## A
Indicar en el informe (en .pdf) el resultado de la
solución encontrada (valor de “x”) si se ejecutan los 3 algoritmos un total
de 30 lanzamientos cada uno. Los parámetros de los algoritmos son:

- Selección por Ranking, Ruleta y Torneo
- Intervalo de la variable de decisión: [-31, 31]∈R (con un dígito decimal)
- Aplicar elitismo: Si (solo en el método Ruleta y Ranking)
- Gen de cruza monopunto aleatorio
- Probabilidad de cruce 0.85
- Probabilidad de mutación 0.09
- Tamaño de la población: 4
- Generaciones: 10

In [3]:
import random
import matplotlib.pyplot as plt

# Parámetros generales
TAMANIO_POBLACION = 4
LONGITUD_CROMOSOMA = 10
TASA_MUTACION = 0.09
TASA_CRUCE = 0.85
GENERACIONES = 10
X_MIN = -31
X_MAX = 31
EPSILON = 0.001  # Valor pequeño para evitar división por cero en la función de aptitud

# Función para convertir binario a decimal
def binario_a_decimal(cromosoma):
    decimal = int(cromosoma, 2)
    x = X_MIN + decimal * (X_MAX - X_MIN) / ((2 ** LONGITUD_CROMOSOMA) - 1)
    return x

# Función objetivo x^2
def funcion_objetivo(x):
    return x ** 2

# Función de aptitud
def aptitud(cromosoma):
    x = binario_a_decimal(cromosoma)
    return 1 / (funcion_objetivo(x) + EPSILON)

# Inicializar población
def inicializar_poblacion(tamanio_poblacion, longitud_cromosoma):
    poblacion = []
    for _ in range(tamanio_poblacion):
        cromosoma = ''.join(str(random.randint(0, 1)) for _ in range(longitud_cromosoma))
        poblacion.append(cromosoma)
    return poblacion

# Selección por ruleta
def seleccion_ruleta(poblacion):
    aptitud_total = sum(aptitud(cromosoma) for cromosoma in poblacion)
    probabilidades = [aptitud(cromosoma) / aptitud_total for cromosoma in poblacion]
    return random.choices(poblacion, weights=probabilidades, k=len(poblacion))

# Selección por torneo
def seleccion_torneo(poblacion, tamanio_torneo=3):
    progenitores = []
    for _ in range(len(poblacion)):
        candidatos = random.sample(poblacion, tamanio_torneo)
        progenitor = max(candidatos, key=aptitud)
        progenitores.append(progenitor)
    return progenitores

# Selección por ranking lineal
def seleccion_ranking(poblacion):
    aptitudes = [(individuo, aptitud(individuo)) for individuo in poblacion]
    aptitudes_ordenadas = sorted(aptitudes, key=lambda x: x[1])
    probabilidades = [(2 * i) / (len(poblacion) * (len(poblacion) - 1)) for i in range(len(poblacion))]
    return [individuo for individuo, _ in random.choices(aptitudes_ordenadas, weights=probabilidades, k=len(poblacion))]

# Cruce monopunto
def cruce_mono_punto(progenitor1, progenitor2, tasa_cruce):
    if random.random() < tasa_cruce:
        punto_cruce = random.randint(1, len(progenitor1) - 1)
        descendiente1 = progenitor1[:punto_cruce] + progenitor2[punto_cruce:]
        descendiente2 = progenitor2[:punto_cruce] + progenitor1[punto_cruce:]
        return descendiente1, descendiente2
    return progenitor1, progenitor2

# Mutación
def mutacion(cromosoma, tasa_mutacion):
    return ''.join(str(int(not int(bit))) if random.random() < tasa_mutacion else bit for bit in cromosoma)

# Algoritmo Genético
def algoritmo_genetico(tamanio_poblacion, longitud_cromosoma, tasa_mutacion, tasa_cruce, generaciones, metodo_seleccion):
    poblacion = inicializar_poblacion(tamanio_poblacion, longitud_cromosoma)
    
    for generacion in range(generaciones):
        #print(f"Generación: {generacion + 1}")
        
        # Selección de progenitores según el método
        if metodo_seleccion == "ruleta":
            progenitores = seleccion_ruleta(poblacion)
        elif metodo_seleccion == "torneo":
            progenitores = seleccion_torneo(poblacion)
        elif metodo_seleccion == "ranking":
            progenitores = seleccion_ranking(poblacion)
        else:
            raise ValueError("Método de selección no válido.")
        
        # Cruce
        descendientes = []
        for i in range(0, len(progenitores), 2):
            descendiente1, descendiente2 = cruce_mono_punto(progenitores[i], progenitores[i + 1], tasa_cruce)
            descendientes.extend([descendiente1, descendiente2])
        
        # Mutación
        descendientes_mutados = [mutacion(descendiente, tasa_mutacion) for descendiente in descendientes]
        
        # Actualizar población
        poblacion = descendientes_mutados
        
        # Mostrar el mejor individuo de la generación
        mejor_individuo = min(poblacion, key=aptitud)
        #print(f"Mejor individuo: {binario_a_decimal(mejor_individuo)} Aptitud: {aptitud(mejor_individuo)}")
        #print("_________________________________________________________________________________")

    mejor_solucion = min(poblacion, key=aptitud)
    return binario_a_decimal(mejor_solucion), funcion_objetivo(binario_a_decimal(mejor_solucion))

# Ejecución del algoritmo
metodo = "ruleta"
mejor_x, mejor_fitness = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, metodo)
print(f"\nMejor solución utilizando el método {metodo}: x = {mejor_x}, Fitness = {mejor_fitness}")

metodo = "torneo"
mejor_x, mejor_fitness = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, metodo)
print(f"\nMejor solución utilizando el método {metodo}: x = {mejor_x}, Fitness = {mejor_fitness}")

metodo = "ranking"
mejor_x, mejor_fitness = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, metodo)
print(f"\nMejor solución utilizando el método {metodo}: x = {mejor_x}, Fitness = {mejor_fitness}")



Mejor solución utilizando el método ruleta: x = 1.7272727272727266, Fitness = 2.983471074380163

Mejor solución utilizando el método torneo: x = -28.03030303030303, Fitness = 785.6978879706153

Mejor solución utilizando el método ranking: x = 31.0, Fitness = 961.0


In [5]:
import pandas as pd

def ejecutar_30_veces():
    datos = []

    for i in range(30):
        # Ranking
        mejor_x_ranking, fitness_ranking = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, "ranking")
        
        # Ruleta
        mejor_x_ruleta, fitness_ruleta = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, "ruleta")
        
        # Torneo
        mejor_x_torneo, fitness_torneo = algoritmo_genetico(TAMANIO_POBLACION, LONGITUD_CROMOSOMA, TASA_MUTACION, TASA_CRUCE, GENERACIONES, "torneo")
        
        # Agregar resultados a la lista de datos
        datos.append([i + 1, 'Ranking', mejor_x_ranking, fitness_ranking])
        datos.append([i + 1, 'Ruleta', mejor_x_ruleta, fitness_ruleta])
        datos.append([i + 1, 'Torneo', mejor_x_torneo, fitness_torneo])

    # Crear DataFrame
    df_resultados = pd.DataFrame(datos, columns=["Lanzamiento", "Algoritmo", "Mejor x", "Aptitud (x²)"])
    return df_resultados

# Ejecutar y mostrar resultados
df_resultados = ejecutar_30_veces()
print(df_resultados)

    Lanzamiento Algoritmo    Mejor x  Aptitud (x²)
0             1   Ranking -16.151515    260.871442
1             1    Ruleta -29.424242    865.786042
2             1    Torneo  30.090909    905.462810
3             2   Ranking -10.030303    100.606979
4             2    Ruleta   6.696970     44.849403
..          ...       ...        ...           ...
85           29    Ruleta  -7.787879     60.651056
86           29    Torneo  -2.878788      8.287420
87           30   Ranking  13.848485    191.780533
88           30    Ruleta -25.909091    671.280992
89           30    Torneo -30.696970    942.303949

[90 rows x 4 columns]


## B 
Completar la siguiente tabla en base a las 30 ejecuciones con los parámetros señalados.

In [6]:
# Calcular métricas estadísticas
def calcular_metricas(df, algoritmo):
    df_algoritmo = df[df['Algoritmo'] == algoritmo]
    
    metricas = {
        "Mínimo": df_algoritmo["Mejor x"].min(),
        "Promedio": df_algoritmo["Mejor x"].mean(),
        "Máximo": df_algoritmo["Mejor x"].max(),
        "Desv. Est.": df_algoritmo["Mejor x"].std()
    }
    
    return pd.Series(metricas, name=algoritmo)

# Calcular métricas para cada algoritmo
metricas_ranking = calcular_metricas(df_resultados, "Ranking")
metricas_ruleta = calcular_metricas(df_resultados, "Ruleta")
metricas_torneo = calcular_metricas(df_resultados, "Torneo")

# Mostrar los resultados y las métricas
print(df_resultados)
print("\nMétricas estadísticas:")
print(pd.DataFrame([metricas_ranking, metricas_ruleta, metricas_torneo]))

    Lanzamiento Algoritmo    Mejor x  Aptitud (x²)
0             1   Ranking -16.151515    260.871442
1             1    Ruleta -29.424242    865.786042
2             1    Torneo  30.090909    905.462810
3             2   Ranking -10.030303    100.606979
4             2    Ruleta   6.696970     44.849403
..          ...       ...        ...           ...
85           29    Ruleta  -7.787879     60.651056
86           29    Torneo  -2.878788      8.287420
87           30   Ranking  13.848485    191.780533
88           30    Ruleta -25.909091    671.280992
89           30    Torneo -30.696970    942.303949

[90 rows x 4 columns]

Métricas estadísticas:
            Mínimo  Promedio     Máximo  Desv. Est.
Ranking -30.030303  0.161616  31.000000   20.832690
Ruleta  -30.818182 -5.224242  31.000000   18.646654
Torneo  -30.939394 -4.064646  30.090909   19.247025


## C
Explicar (en el .pdf) una interpretación de los resultados obtenidos en el ítem anterior.

In [7]:
# Ejecutar pruebas con diferentes combinaciones de parámetros
def probar_combinaciones():
    resultados = []
    poblaciones = [4, 8, 16]  # Tamaño de la población
    generaciones = [10, 20, 50]  # Número de generaciones
    tasas_mutacion = [0.01, 0.05, 0.1, 0.2]  # Probabilidad de mutación
    
    for pop_size in poblaciones:
        for gen_count in generaciones:
            for mut_rate in tasas_mutacion:
                for metodo in ["ranking", "ruleta", "torneo"]:
                    mejor_x, mejor_fitness = algoritmo_genetico(pop_size, LONGITUD_CROMOSOMA, mut_rate, TASA_CRUCE, gen_count, metodo)
                    resultados.append([metodo, pop_size, gen_count, mut_rate, mejor_x, mejor_fitness])
    
    # Convertir resultados a DataFrame
    df_resultados = pd.DataFrame(resultados, columns=["Algoritmo", "Tamaño de Población", "Generaciones", "Pm", "Mejor x", "Aptitud (x²)"])
    return df_resultados

# Ejecutar las pruebas
df_resultados = probar_combinaciones()

# Encontrar la mejor combinación por algoritmo
mejores_combinaciones = df_resultados.loc[df_resultados.groupby('Algoritmo')['Aptitud (x²)'].idxmax()]

# Mostrar resultados
print(df_resultados)
print("\nMejores combinaciones por algoritmo:")
print(mejores_combinaciones)

    Algoritmo  Tamaño de Población  Generaciones    Pm    Mejor x  \
0     ranking                    4            10  0.01  10.939394   
1      ruleta                    4            10  0.01  17.363636   
2      torneo                    4            10  0.01   0.090909   
3     ranking                    4            10  0.05  27.000000   
4      ruleta                    4            10  0.05   8.878788   
..        ...                  ...           ...   ...        ...   
103    ruleta                   16            50  0.10  26.757576   
104    torneo                   16            50  0.10  30.515152   
105   ranking                   16            50  0.20 -26.454545   
106    ruleta                   16            50  0.20  30.757576   
107    torneo                   16            50  0.20  30.878788   

     Aptitud (x²)  
0      119.670340  
1      301.495868  
2        0.008264  
3      729.000000  
4       78.832874  
..            ...  
103    715.967860  
104    931.